# Bonus tutorial: intro

Welcome to the bonus section on the **inhibitory plasticity**! Here, we will consider one postsynaptic neuron which receives $N$ presynaptic inputs (both excitatory and inhibitory). We start by visualizing how the inhibitory plasticity shapes the connections between the inhibitory presynaptic neurons and the postsynaptic neuron. Then, we will deepen the analysis by comparing the dynamics in the two cases of no plasticity and inhibitory plasticity.

Please, just run the two cells below, Initialization and Utility functions.

## Initialization

In [ ]:
!pip install numpy scipy matplotlib ipywidgets scikit-learn --quiet
import numpy as np
import scipy.linalg as lin
from numpy.random import default_rng
rng = default_rng()
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
plt.style.use("https://github.com/comp-neural-circuits/plasticity-workshop/raw/dev/plots_style.txt")

## Utility functions

In [ ]:
def nonrandom_spike_trains(rho,time_total,DeltaT,discretization_step=1E-4):
    """
    Generates presynaptic and postsynaptic spike trains characterised by a number of pairs 
    of pre- and post-synaptic spikes with time distance `DeltaT`.
    The pairs are repeated at frequency `rho` 
    
    Parameters:
    rho (number) : the firing frequency in Hz of each neuron
    time_total (number) : the duration of the spike trains, in seconds
    DeltaT (number) : the time distance between pre and post spike, in seconds  
    discretization_step = 1E-4    : a very small time interval (in seconds), express the spike trains as binary. 
                                     The default is to consider time-bins of 0.1 ms.                        
    
    Returns:
    times  (1D numpy array) : times associated to binary spiketrain (in seconds)
    spiketrain_pre (1D numpy array) : binary array with the first spiketrain (one element is a bin of size `discretization_step`)
    spiketrain_post (1D numpy array) : binary array with the second spiketrain
    """
    
    assert DeltaT < (1/rho) , "interval between spikes is too large compared to frequency !"
    
    # time vector
    times = np.arange(0.0,time_total,discretization_step)
    
    # spike times, with frequency rho
    train1_s = np.arange(0.0,time_total,1/rho) 
    train2_s = train1_s + abs(DeltaT) # second neuron fires with DeltaT offset 
    
    # spike times in discretized units, converted to integer
    train1_discr = (train1_s * (1/discretization_step)).astype(np.int64) 
    train2_discr = (train2_s * (1/discretization_step)).astype(np.int64)
    
    # build the binary spike trains, using discretized spike times
    train1_bin = np.zeros(len(times))
    train1_bin[train1_discr] = 1
    train2_bin = np.zeros(len(times))
    train2_bin[train2_discr] = 1
    # sign of DeltaT determines which spikes first
    if DeltaT > 0:
        # pre spike before, post spike later
        return times,train1_bin,train2_bin
    else:
        # post spike before, pre spike later
        return times,train2_bin,train1_bin
    
def Poisson_spike_train(discretization_step, t_range, rate, myseed=False):
    """
    Generates a Poisson spike train
    
    Parameters:
    discretization_step   (number) : discretization time step [s]
    t_range       (1D numpy array) : time interval [s]
    rate                  (number) : intensity of the Poisson process [Hz]
    myseed                (number) : random seed. int or boolean

    Returns:
    Poisson_train (1D numpy array) : a spike train of binary values (1 if spike, 0 otherwise)
    """
    # set random seed
    if myseed:
        np.random.seed(seed=myseed)
    else:
        np.random.seed()

    # generate an uniformly distributed random variable (for the ISI = interspike interval)
    unirnd = np.random.rand(len(t_range))

    # generate Poisson train
    # note that we divide by 1000 in the expression below because the time is measured in ms, 
    # whereas the intensity is given in Hz (s^-1)
    poisson_train = 1. * (unirnd < rate * (discretization_step))

    return poisson_train

    
def inhSTDP_analytic(eta,rho,tau,Delta_t):
    """    
    Computes the change in weight for a single pre-post pair due to inhibitory STPD
    
    Parameters:
    tau (number) : plasticity timescale
    eta (number) : learning rate 
    rho (number) : target rate
    Delta_t (1D numpy array) : differences between time post spike and time pre spike
    
    Returns 
    DeltaW (1D numpy array) : the weight change due to the inhibitory STDP rule
                              for each time differenence in tau
    """
    # Calculate DeltaW
    alpha = 2*tau*rho
    DeltaW = eta * ( (1/(2*tau)) * np.exp(-np.abs(Delta_t)/tau) - rho)
    return DeltaW
    
def inhibitory_stdp_interactive_plot(eta=1.0,rho=1.0,tau=20E-3):
    rate_protocol = 0.5
    xmax = 0.5
    dplot = xmax/30
    x = np.arange(-xmax,xmax,dplot)
    y = np.fromiter(map(lambda _x : inhibitory_stdp_pairing(_x,rate_protocol,5.,tau,eta,rho)[2] , x),dtype=np.float64)
    # let's add the analytic! 
    y_an = inhSTDP_analytic(eta,rho,tau,x)
    # analyti refers to one pair only, but numeric depends on rate of presentation
    y_an *= rate_protocol
    
    x *= 1000 # switch to ms
    xmax *= 1000 # switch to ms
    plt.figure()
    plt.plot([-xmax, xmax], [0, 0], 'k', linestyle=':')
    #plt.plot([0, 0], [-80, 100], 'k', linestyle=':')
    plt.plot(x, y, 'r')
    plt.plot(x,y_an, 'y--')
    plt.xlabel(r'$\Delta t=$ t$_{\mathrm{post}}$ - t$_{\mathrm{pre}}$ (ms)',fontsize=16)
    plt.ylabel(r'$\Delta $W ',fontsize=16)
    plt.title('The inhibitory STDP rule', fontsize=18, fontweight='bold')
    plt.show()
    return
    
def inhSTDP_plot(eta, rho, tau):
    """    
    Generates a plot for the inhibitory plasticity rule
    
    Parameters:
    tau (number) : plasticity timescale
    eta (number) : learning rate 
    rho (number) : target rate
    
    Returns:
    A plot for the inhibitory STDP rule
    """
    Delta_t =  np.linspace(-200e-3, 200e-3, 100)
    # Calculate the synaptic change via the specific function
    DeltaW = inhSTDP_analytic(eta, rho, tau,Delta_t)
    
    # Plotting
    plt.figure(figsize=(8,5))
    plt.plot([Delta_t[0], Delta_t[-1]], [0, 0], 'k', linestyle=':')
    plt.plot([0, 0], [-0.001, 0.0015], 'k', linestyle=':')
    plt.plot(Delta_t, DeltaW, 'r')
    plt.xlabel(r'$\Delta t=$ t$_{\mathrm{post}}$ - t$_{\mathrm{pre}}$ (ms)',fontsize=16)
    plt.ylabel(r'$\Delta $W (%)',fontsize=16)
    plt.title('The inhibitory STDP rule', fontsize=18, fontweight='bold')
    plt.show()

    
def inh_analysis_plot_noplasticity(num_small_intervals,duration,post_spike,Exkeep,Inkeep):
    """    
    Generates a plot for the postsynaptic firing rate and for the currents 
                (excitatory, inhibitory, net) in the absence of plasticity
    
    Parameters:
    num_small_intervals (number) : number of intervals over which calculate the average value of the functions
    duration (number) : time duration of each small interval [ms] 
    post_spike (array) : postsynaptic binary spike train
    Exkeep (array) : excitatory synaptic current over time
    Inkeep (array) : inhibitory synaptic current over time
    
    Returns:
    Plots for the temporal evolution of postsynaptic firing rate and the currents (excitatory, inhibitory, net)
    """
    
    post_rate = np.zeros(num_small_intervals)
    avg_exkeep = np.zeros(num_small_intervals)
    avg_inkeep = np.zeros(num_small_intervals)
    
    for i in range(num_small_intervals):
        # calculate the mean value of postsynaptic firing rate and synaptic currents over a small interval
        post_rate[i] = np.sum(post_spike[i*duration:(i+1)*duration])/ duration * 1000   # *1000 to convert in Hz
        avg_exkeep[i] = np.mean(Exkeep[i*duration:(i+1)*duration]) 
        avg_inkeep[i] = np.mean(Inkeep[i*duration:(i+1)*duration])
    # plotting
    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,12));
    ax1.set_ylim(15,20) 
    ax1.plot(range(num_small_intervals),post_rate)
    ax1.set_xlabel("time (s)", fontsize=12);
    ax1.set_ylabel("Firing rate [Hz]", fontsize=12);
    ax1.set_title("Postsynaptic firing rate", fontsize=14);
    ax2.plot(range(num_small_intervals), avg_exkeep/100,'r',linewidth=3, label= 'Excitatory synaptic current')
    ax2.plot(range(num_small_intervals), avg_inkeep/100, 'g', linewidth=3, label= 'Inhibitory synaptic current')
    ax2.plot(range(num_small_intervals), (avg_inkeep + avg_exkeep)/100, 'k', linewidth=3,label= 'Net synaptic current')
    ax2.legend(fontsize = 10)
    ax2.set_xlabel("time (s) ", fontsize=12);
    ax2.set_ylabel('Currents', fontsize=12);
    ax2.set_title("Currents before plasticity", fontsize=14);
    
    
def inh_analysis_plot_withplasticity(num_small_intervals,post_spike,save_inh_weights, duration,Exkeep,Inkeep):
    """    
    Generates a plot for the postsynaptic firing rate and a plot for 
    the currents (excitatory, inhibitory, net) in the presence of inhibitory plasticity
    
    Parameters:
    num_small_intervals (number) : number of intervals over which calculate the average value of the functions
    duration (number) : time duration of each small interval [ms] 
    post_spike (array) : postsynaptic binary spike train
    save_inh_weights (array) : mean values for the inhibitory synaptic weights for each time
    Exkeep (array) : excitatory synaptic current over time
    Inkeep (array) : inhibitory synaptic current over time
    
    Returns:
    Plots for the temporal evolution of :
    postsynaptic firing rate, 
    average inhibitory synaptic weight,
    currents (excitatory, inhibitory, net)
    """
    post_rate = np.zeros(num_small_intervals)
    inh_weights = np.zeros(num_small_intervals)
    avg_exkeep = np.zeros(num_small_intervals)
    avg_inkeep = np.zeros(num_small_intervals)
    
    for i in range(num_small_intervals):
        post_rate[i] = np.sum(post_spike[i*duration:(i+1)*duration])/ duration * 1000
        inh_weights[i] = np.mean(save_inh_weights[i*duration:(i+1)*duration])
        avg_exkeep[i] = np.mean(Exkeep[i*duration:(i+1)*duration]) 
        avg_inkeep[i] = np.mean(Inkeep[i*duration:(i+1)*duration])
    fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,12));
    ax1.plot(range(num_small_intervals),post_rate)
    ax1.set_xlabel("time (s)", fontsize=12);
    ax1.set_ylabel("Firing rate [Hz]", fontsize=12);
    ax1.set_title("Postsynaptic firing rate", fontsize=14);
    ax2.plot(range(num_small_intervals),inh_weights)
    ax2.set_xlabel("time (s) ", fontsize=12);
    ax2.set_ylabel("Mean inhibitory weights", fontsize=12);
    ax2.set_title("Inhibitory weights", fontsize=14);
    ax3.plot(range(num_small_intervals), avg_exkeep/100,'r',linewidth=3, label= 'Excitatory synaptic current')
    ax3.plot(range(num_small_intervals), avg_inkeep/100, 'g', linewidth=3, label= 'Inhibitory synaptic current')
    ax3.plot(range(num_small_intervals), (avg_inkeep + avg_exkeep)/100, 'k', linewidth=3,label= 'Net synaptic current')
    ax3.legend(fontsize = 10)
    ax3.set_xlabel("time (s) ", fontsize=12);
    ax3.set_ylabel('Currents', fontsize=12);
    ax3.set_title("Currents after plasticity", fontsize=14);

# Inhibitory plasticity

We now consider synaptic plasticity *from* inhibitory neurons *to* one excitatory neurons. Careful here! A stronger (positive) synaptic weight means that our output neuron is inhibited *more*.



The STDP rule used for inhibition is symmetric, positive for short time differences between pre and post spikes, and negative as the difference increase.

$$
\Delta W = \eta \left( \frac{1}{2\tau}\, 
\exp\left(-\frac{\Delta t}{\tau}\right) - \alpha\right) \quad
\text{and} \quad
\alpha = 2\,\rho\,\tau
$$

## Online update with inhibitory STPD

Like we did before, we will express the plasticity rule in terms of spike detectors. Let's consider a p**R**esynaptic detector $r(t)$ and a p**O**st-synaptic detector $o(t)$. Therefore:

\begin{align}
\frac{\mathrm d\,r}{\mathrm d \,t} &= -  \frac{r}{\tau} \quad ; \quad 
\text{if presynaptic neuron fires at time $t^*$ :} \quad r(t^*) \leftarrow r+1 \quad \\
\frac{\mathrm d\,o}{\mathrm d \,t} &= -  \frac{o}{\tau} \quad ; \quad 
\text{if postsynaptic neuron fires at time $t^*$ :} \quad o(t^*) \leftarrow o+1
\end{align}

If there are multiple presynaptic neurons $(1,2,\ldots,N)$ we need to consider a separate trace for each of them. Therefore $(r_1(t),r_2(t),\ldots r_N(t))$ ... but let's think about that later.

The online inhibitory STDP update ( [Vogels et al, 2011](http://www.sciencemag.org/cgi/doi/10.1126/science.1211095) , supplement)  is defined as follows:

\begin{align} 
W \leftarrow & W + \eta \; (o(t^*) - \alpha) \quad 
&& \text{for *presynaptic* spike at time $t^*$}\\
W \leftarrow & W + \eta \; r(t^*) \quad 
&& \text{for *postsynaptic* spike at time $t^*$}\\
\end{align}

Where $\tau$ is the time constant associated to the plasticity window, $\eta$ is the learning constant, and the parameter $\alpha$ can be reparametrized as $\alpha = 2\;\tau\;\rho$, where $\rho$ here represents a "target rate".

We will now implement this rule for a given pre and post spiketrain.

### **Exercise :** implement the rule for fixed spiketrains 

In [ ]:
interact(inhSTDP_plot, eta = fixed(1e-4), rho = fixed(0.1), tau = (5e-3,70e-3,1e-4));

## Online update with inhibitory STPD

Like we did before, we will express the plasticity rule in terms of spike detectors. Let's consider a p**R**esynaptic detector $r(t)$ and a p**O**st-synaptic detector $o(t)$. Therefore:

\begin{align}
\frac{\mathrm d\,r}{\mathrm d \,t} &= -  \frac{r}{\tau} \quad ; \quad 
\text{if presynaptic neuron fires at time $t^*$ :} \quad r(t^*) \leftarrow r+1 \quad \\
\frac{\mathrm d\,o}{\mathrm d \,t} &= -  \frac{o}{\tau} \quad ; \quad 
\text{if postsynaptic neuron fires at time $t^*$ :} \quad o(t^*) \leftarrow o+1
\end{align}

If there are multiple presynaptic neurons $(1,2,\ldots,N)$ we need to consider a separate trace for each of them. Therefore $(r_1(t),r_2(t),\ldots r_N(t))$ ... but let's think worry about that later.

The online inhibitory STDP update ( [Vogels et al, 2011](http://www.sciencemag.org/cgi/doi/10.1126/science.1211095) , supplement)  is defined as follows:

\begin{align} 
W \leftarrow & W + \eta \; (o(t^*) - \alpha) \quad 
&& \text{for *presynaptic* spike at time $t^*$}\\
W \leftarrow & W + \eta \; r(t^*) \quad 
&& \text{for *postsynaptic* spike at time $t^*$}\\
\end{align}

Where $\tau$ is the time constant associated to the plasticity window, $\eta$ is the learning constant, and the parameter $\alpha$ can be reparametrized as $\alpha = 2\;\tau\;\rho$, where $\rho$ here represents a "target rate".

We will now implement this rule for a given pre and post spiketrain.

### **Exercise :** implement the rule for fixed spiketrains 

In [ ]:
def inhibitory_stpd_weight_change(train_post,train_pre,tau,eta,rho,discretization_step=1E-4,wstart=0.0):
    """
    Compute the weight change over time according to the inhibitory STDP, for given spike trains.
    (imagine the weight change in an experiment where neurons are forced to fire)
    
    Parameters:
    train_post (1D numpy array) : binary vector with spike train. Each bin has duration `discretization_step`
    train_pre  (1D numpy array) : same but for presynaptic neuron
    tau (number) : plasticity time constant
    eta (nunber) : plasticity rate
    rho (number) : plasticity target-rate
    discretization_step = 1E-4 : size of single time bin, in seconds
    wstart = 0.0  :  weight at t=0 . Note that here the weight does not influence the spiking dynamics in any way
    
    Return
    times (1D numpy array) : -
    w     (1D numpy array) : w[k] is the pre to post synaptic weight at time times[k]  
    """
    
    # compute total time, and generate vector of timesteps
    Ntimes = len(train_post)
    Tend = Ntimes * discretization_step 
    times = np.arange(0,Tend,discretization_step)
    
    # express parameter alpha as a function of tau and rho
    alpha = 2*tau*rho
    
    # Initialize the synaptic weight
    w = np.empty(Ntimes)
    w[0] = wstart
    # Initialize the pre- and postsynaptic event detectors
    r = np.empty(Ntimes)     # presynaptic event detector
    r[0] = 0.0
    o = np.empty(Ntimes)      # postsynaptic event detector
    o[0] = 0.0
    
    w_last = wstart # value of last weight computed 
    
    for i in range(Ntimes):
        w[i] = w_last
        # >>> EXERCISE 
        # if there is a pre-synaptic spike  ... 
        # 1. update the pre-synaptic detector r[i]
        # 2. updated the weight using the postsynaptic trace o[i]
        
        if train_pre[i]==1:     
            r[i] += 1
            w_last += eta*(o[i]-alpha)
        # <<<
        
        # >>> EXERCISE
        # if there is a post-synaptic spike ...
        # 1. update the post-synaptic detector o[i]
        # 2. updated the weight using the presynaptic trace r[i]
        if train_post[i]==1:
            o[i] += 1
            w_last += eta*r[i] 
         
        # Euler step to update traces
        if i < Ntimes-1 :   # cannot update after last timestep
            # >>> EXERCISE 
            # Apply Euler's method to the detector equations
            r[i+1] = r[i] - discretization_step * r[i]/tau
            o[i+1] = o[i] - discretization_step * o[i]/tau
            # <<<
    return times,w

The function below, already given, generates two spike trains with fixed frequencies and a pre-post time difference, as has been done before. This time, however, we apply the inhibitory STDP rule to it. In essence, this simulates the weight change in an hypothetical experiment with a pre-post spiking paradigm.


In [ ]:
def inhibitory_stdp_pairing(DeltaT,rate_protocol,duration_protocol,tau,eta,rho,discretization_step=1E-4):
    
    # generate spike trains with pre-post training protocol
    times,train1,train2 = nonrandom_spike_trains(rate_protocol,duration_protocol,DeltaT,
                                                          discretization_step = discretization_step)
    
    # compute weight change
    timesw, w = inhibitory_stpd_weight_change(train2,train1,tau,eta,rho, 
                                              discretization_step=discretization_step,wstart=0.0)           
                   
    # average change of weight per second                                           
    w_mean_change = w[-1]/times[-1]
    return timesw,w, w_mean_change
                                              
    

In [ ]:
interact_manual(inhibitory_stdp_interactive_plot,eta =fixed(1), rho = (0.1,10,0.1) , tau = (5E-3,100E-3,1E-3));

## Dynamics...

In this section, we will understand how inhibitory synaptic plasticity balances excitation and inhibition. We consider $N$ presynaptic neurons, $80%$ of which are excitatory and the rest are inhibitory. The dynamics of these neurons follows the Poisson distribution.

The general framework of the following is taken from [Vogels et al, 2011](http://www.sciencemag.org/cgi/doi/10.1126/science.1211095): the model used for the postsynaptic neuron is a leaky integrate-and-fire neuron, characterized by a time constant, $ \tau= 20$ ms, and a resting membrane potential, $V_{rest} = -60$ mV. Whenever the membrane potential crosses a spiking threshold of $V_\theta = −50 $mV, an action potential is generated and the membrane potential is reset to the resting potential, where it remains clamped for a $\tau_{ref}= 5$ ms refractory period (i.e. the neuron cannot spike over this period).  The subthreshold membrane voltage $V_i(t)$ obeys:

$$
\tau \frac{dV_i}{dt} = (V_{rest} - V_i) - g_i^E V_i + g_i^I( V^I - V_i ) \times \frac{1}{g^{leak}}
$$

where $V_I = -80$ mV is the reversal inhibitory potential, $g_i^E$ and $g_i^I$ are the excitatory and inhibitory synaptic conductance respectively, and $g^{leak}$ is a constant, leak conductance.
When the neuron receives a presynaptic action potential, the postsynaptic conductance is increased

$$
g_i^E \rightarrow g_i^E + \bar g_E W_i \text{ for an excitatory spike and } 
$$

$$
g_i^I \rightarrow g_i^I + \bar g_I W_i \text{ for an inhibitory spike} 
$$

where $\bar g $ is a constant and $W_i$ is the synaptic weight from neuron $i$ to the postsynaptic neuron.

Otherwise, these parameters obey the equations

$$
\tau_E \frac{dg_i^E}{dt} = -g_i^E\text{  and  } \tau_I \frac{dg_i^I}{dt} = -g_i^I
$$
with synaptic time constants $\tau_E = 5$ ms and $\tau_I = 10$ ms.

We will implement the temporal evolution of the postsynaptic firing rate, and the total excitatory, inhibitory, and net membrane currents, first in the absence of plasticity and then with inhibitory synaptic plasticity. In the latter case, we will keep track of each spike and we will update the inhibitory synaptic weights, following the same procedure described above in section **2.1.**
When analysing this case, we will see also how the average synaptic weights of the inhibitory synapses evolve over time. 

In the next cell, a function to update membrane potential is provided. You can have a look at it or you can run it and move on to the next exercise.

**Helper function**

In [ ]:
def V_update(t,tolos,tRef, V_old, VRest, gLeak, gExc, gInh, taumem, EGABA):
    '''
    Updates the membrane potential for the postsynaptic neuron.
    
    Parameters:
    t (number) : running time instant
    tolos (number) : T(ime)O(f)L(ast)p(O)synaptic(S)pike (Keeping track of the postsynaptic cell’s refractory period )
    V_old : the current value of the membrane potential
    VRest, gLeak, taumem, EGABA: parameters for the model (see below for details)
    gExc, gInh : synaptic conductances 
    
    Return
    V_new : the updated value for the membrane potential 
    '''
    
    if ((t - tolos) < tRef):  # if the cell is refractory, keep V at Vrest
        V_new = VRest
    else:                     # if the cell is not refractory
        gTot = gLeak + gExc + gInh   # calculate the total membrane conductance
        tauEff = taumem/gTot         # the effective time constant
        VInf = ((gLeak*VRest + gInh*EGABA)/gTot)    # the membrane potential that V strives towards
        V_new = VInf + (V_old - VInf)*np.exp(-dt/tauEff)   # update the membrane potential
    return V_new

### ...without inhibitory plasticity

In the following code, your task will be to understand how the synaptic conductance is updated at each time instant and to implement what happens if there is a spike in the postsynaptic neuron.


In [ ]:
# Parameters 
dt = 0.1    # discretization step in ms
duration=1000 # duration of a small interval in ms (== 1 s)
num_small_intervals=60   # number of the intervals over which will calculate the average value of the functions
range_t=np.arange(0,num_small_intervals*duration,dt)   # the whole time interval 
rate = 60  # presynaptic firing rate
n = 100    # number of inputs
exc = int(0.8 * n)  # fraction of excitatory inputs
tauExc = 5  # excitatory synaptic time constant, in ms
tauInh = 10  # inhibitory synaptic time constant, in ms
gBarExc = 0.14 # scaling factor of the excitatory synaptic conductance
gBarInh = 0.35 # scaling factor of the inhibitory synaptic conductance
VRest=-60   # resting potential in mV
Vth=-50     # threshold in mV
taumem=20   # membrane time constant, in ms
EGABA=-80   # inhibitory reversal potential, in mV
eta = 0.01   # plasticity Rate
alpha = 0.25 * eta   # depression factor
tRef = 5 # refractory period for the spike trains
gLeak = 1   # leak conductance

# set the synaptic weights (they will be fixed)
W=np.zeros(n)
np.random.seed(7)
W[0:exc] =  (1.1 + np.random.random(exc))   # the excitatory synapses are much stronger than the inhibitory
W[exc:n] = 0.1

# generate the input spike trains under the Poisson distribution
input_spiketrains = np.zeros((n, len(range_t)))
for s in range(n):
    input_spiketrains[s,:] = Poisson_spike_train(dt, range_t, rate/1000, s*100)
post_spike = np.zeros(len(range_t))

# initialization 
gExc = 0 # excitatory synaptic conductance
gInh = 0 # inhibitory synaptic conductance
tolos = 0  # T(ime)O(f)L(ast)p(O)synaptic(S)pike (keeping track of the postsynaptic cell’s refractory period )
V = np.zeros(len(range_t)) # vector to keep the membrane potential over time
V[0]=VRest  # membrane potential is initially at VRest.

# vectors to keep the synaptic currents
Exkeep= np.zeros(len(range_t)) 
Inkeep = np.zeros(len(range_t))

for ind,t in enumerate(range_t):
    if ind == 0:
        continue
    # >>> EXERCISE 
    # 1. Apply Euler's method to the conductance equations 
    # (in this case, we need only to update the values, without saving them over time)
    gExc = gExc - dt/tauExc * gExc 
    gInh = gInh - dt/tauInh * gInh
    # <<< 
    for s in range(n):
        if input_spiketrains[s, ind] == 1:
            if s<exc:
                gExc += (gBarExc * W[s])
            else:
                gInh = gInh + (gBarInh * W[s])
    # postsynaptic neuron 
    V[ind] = V_update(t,tolos,tRef, V[ind-1], VRest, gLeak, gExc, gInh, taumem, EGABA)
    # >>> EXERCISE 
    # if there is a spike in the postsynaptic neuron (how can you know when this is the case?), then...
    # 1. save the postsynaptic spike in the vector post_spike
    # 2. update the time of the last postsynaptic spike (tolos) with the current time instant
    # 3. reset the current membrane potential to Vrest.
    if V[ind] >= Vth:
        tolos = t
        V[ind] = VRest 
        post_spike[ind] = 1
    # <<<
    Exkeep[ind] = gExc * V[ind]          # excitatory synaptic current
    Inkeep[ind] = gInh * (V[ind] - EGABA)   # # inhibitory synaptic current
inh_analysis_plot_noplasticity(num_small_intervals,duration,post_spike,Exkeep,Inkeep)

### ...after inhibitory plasticity

Finally, we compare the results above with the case in which the inhibitory synapses are subject to the plasticity mechanism. The code is an extension of the previous one, and your task will be to implement the plasticity rule when needed. 

After visualizing the results, what differences can you observe with the case of no plasticity?

In [ ]:
dt = 0.1    # discretization step in ms
duration=1000 # duration of a small interval in ms (== 1 s)
num_small_intervals=60   # number of seconds
range_t=np.arange(0,num_small_intervals*duration,dt)   # the whole time interval 
rate = 60  # presynaptic firing rate
n = 100    # number of inputs
exc = int(0.8 * n)  # fraction of excitatory inputs
tauExc = 5  # excitatory synaptic time constant, in ms
tauInh = 10  # inhibitory synaptic time constant, in ms
gBarExc = 0.14 # scaling factor of the excitatory synaptic conductance
gBarInh = 0.35 # scaling factor of the inhibitory synaptic conductance
VRest=-60   # resting potential in mV
Vth=-50     # threshold in mV
taumem=20   # membrane time constant, in ms
EGABA=-80   # inhibitory reversal potential, in mV
eta = 0.01   # plasticity Rate
alpha = 0.25 * eta   # depression factor
tRef = 5 # refractory period for the spike trains
gLeak = 1   # leak conductance

# set the synaptic weights (the inhibitory. are plastic)
W=np.zeros(n)
np.random.seed(7)
W[0:exc] =  (1.1 + np.random.random(exc))   # the excitatory synapses are much stronger than the inhibitory
W[exc:n] = 0.1

# generate the input spike trains under the Poisson distribution
input_spiketrains = np.zeros((n, len(range_t)))
for s in range(n):
    input_spiketrains[s,:] = Poisson_spike_train(dt, range_t, rate/1000, s*100)
post_spike = np.zeros(len(range_t))

# initialization 
gExc = 0 # excitatory synaptic conductance
gInh = 0 # inhibitory synaptic conductance
tolos = 0  # T(ime)O(f)L(ast)p(O)synaptic(S)pike (keeping track of the postsynaptic cell’s refractory period )
V = np.zeros(len(range_t)) # vector to keep the membrane potential over time
V[0]=VRest  # membrane potential is initially at VRest.

# vectors to keep the synaptic currents
Exkeep= np.zeros(len(range_t)) 
Inkeep = np.zeros(len(range_t))


# PLASTICITY
# time window of the learning rule, in ms
tauPlasticity = 20
# initialization
r1 = np.zeros(n)  # vector to save the presynaptic learning trace. 
o1 = 0  # postsynaptic learning trace.
save_inh_weights = np.zeros(len(range_t)) # vector to save the inhibitory weights (the only ones subject to plasticity)


for ind,t in enumerate(range_t):
    if ind == 0:
        continue
    # Plasticity     
    # >>> EXERCISE
    # Apply Euler's method to the detector equations 
    # (recall that there is one postsynaptic trace and there are n inputs, but we need to keep track only of the inhibitory
    o1 = o1 - dt/tauPlasticity * o1
    for s in range(exc,n):
        r1[s] = r1[s] - dt/tauPlasticity * r1[s]
    # <<< 
    
    # Update conductance 
    gExc = gExc - dt/tauExc * gExc 
    gInh = gInh - dt/tauInh * gInh
    for s in range(n):
        if input_spiketrains[s, ind] == 1:
            if s<exc:
                gExc += (gBarExc * W[s])
            else:
                gInh = gInh + (gBarInh * W[s])
                # Plasticity
                # >>> EXERCISE
                # Update the presynaptic trace
                # Update the synaptic weight
                r1[s] += eta                    
                W[s] = W[s] + o1 - alpha    
                # <<<
                if W[s] < 0:
                    W[s] = 0     # this ensure non-negative synaptic weights.
    # postsynaptic neuron 
    V[ind] = V_update(t,tolos,tRef, V[ind-1], VRest, gLeak, gExc, gInh, taumem, EGABA)
    if V[ind] >= Vth:
        post_spike[ind] = 1   # save the postsynaptic spike in the vector post_spike
        tolos = t             # update the time of the last postsynaptic spike (tolos) with the current time instant
        V[ind] = VRest        # reset the current membrane potential to Vrest.
        # plasticity
        # >>> EXERCISE
        # update the postsynaptic detector trace
        # update every inhibitory synapse 
        o1 += eta
        for s in range(exc,n):
            W[s] += r1[s]   
        # <<<    
    Exkeep[ind] = gExc * V[ind]        # excitatory synaptic current
    Inkeep[ind] = gInh * (V[ind] - EGABA)   # inhibitory synaptic current
    save_inh_weights[ind] = np.mean(W[exc:n])  # keep track of the average inhibitory weight for each time

save_inh_weights[0]=save_inh_weights[1]

inh_analysis_plot_withplasticity(num_small_intervals,post_spike,save_inh_weights, duration,Exkeep,Inkeep)

That is the end. **Congratulations for completing the entire plasticity tutorial**! :) we hope that it has been useful, and that you enjoyed it!